In [42]:
import numpy as np
import pandas as pd
from glob import glob
import os
from datetime import datetime

In [ ]:
high_res_folder_name = 'D://FINO1Data/10Hz/'

files = glob(high_res_folder_name + '**/*.txt', recursive=True)

df = pd.DataFrame()
for file in files[0:2]:
    print(file)
    if not file.endswith('.txt'):
        continue
    df = pd.concat([df,pd.read_csv(file, sep=' ')])

df = df.reset_index(drop = True)
df.columns = df.columns.str.replace('(', '_', regex=False)
df.columns = df.columns.str.replace(')', '', regex=False)
df['datetime'] = pd.to_datetime(df['Date'] + ' ' +  df['Time']) 
df = df.drop(columns=['Date','Time'])

if not os.path.exists('../02Data'):
    os.makedirs('../02Data')
    
df.to_pickle('../02Data/10hz_data')
df.columns

D://FINO1Data/10Hz\200601\FINO_MGC__2006_01_01_00_00.txt
D://FINO1Data/10Hz\200601\FINO_MGC__2006_01_01_00_10.txt


Index(['vh_80', 'vh_60', 'vh_40', 'dir_80', 'dir_60', 'dir_40', 'u_80', 'v_80',
       'w_80', 'T_80', 'u_60', 'v_60', 'w_60', 'T_60', 'u_40', 'v_40', 'w_40',
       'T_40', 'datetime'],
      dtype='object')

In [82]:
high_res_folder_name = 'D://FINO1Data/10min/'

files = glob(high_res_folder_name + '**/*.dat', recursive=True)

df = pd.DataFrame()
for file in files[:]:
    parameter = file.split('_')[1]
    altitude = file.split('_')[2]
    if altitude[-1] != 'm':
        altitude = file.split('_')[3][:-1]
    else:
        altitude = file.split('_')[2][:-1]
    col_name = f'{parameter}_{altitude}'
    df_aux = pd.read_csv(file, sep='\t', skiprows=[0,1,2,3,5], encoding='latin1')
    df[col_name] = df_aux['Value']
df['datetime'] = df_aux['Time']
df['datetime'] = pd.to_datetime(df['datetime'])

if not os.path.exists('../02Data'):
    os.makedirs('../02Data')
    
df.to_pickle('../02Data/10min_data')
df.columns

Index(['airpressure_21', 'airpressure_92', 'airtemperature_101',
       'airtemperature_34', 'airtemperature_42', 'airtemperature_52',
       'airtemperature_72', 'globalradiation_34', 'globalradiation_93',
       'precipitation_101', 'precipitation_24', 'relativehumidity_101',
       'relativehumidity_34', 'relativehumidity_42', 'relativehumidity_52',
       'relativehumidity_72', 'uvradiation_93', 'uvradiation_34',
       'winddirection_42', 'winddirection_62', 'winddirection_82',
       'winddirection_34', 'winddirection_51', 'winddirection_71',
       'winddirection_91', 'windspeed_102', 'windspeed_34', 'windspeed_41',
       'windspeed_51', 'windspeed_61', 'windspeed_71', 'windspeed_81',
       'windspeed_91', 'windspeed_42', 'windspeed_62', 'windspeed_82',
       'datetime'],
      dtype='object')

In [163]:
# COMPUTE OBUKHOV LENGTH

# Function to obtain friction velocity
def compute_friction_velocity(u, v, w):
    up = u - np.mean(u)
    vp = v - np.mean(v)
    wp = w - np.mean(w)
    uw = up*wp
    vw = vp*wp
    u_a = (np.mean(uw)**2 + np.mean(vw)**2)**(1/4)

    return u_a

def compute_mixing_ratio(rh, t, p):
    # Convert temperature to Celsius
    t_c = t - 273.15
    
    # Saturation vapor pressure in hPa
    e_s = 6.112 * np.exp(17.67 * t_c / (t_c + 243.5))
    
    # Actual vapor pressure
    e = rh / 100 * e_s
    
    # Mixing ratio
    r = 0.622 * e / (p - e)  # kg/kg
    return r

def compute_virtual_temperature(t, r):
    # Convert temperature to Celsius
    t_v = t*(1+0.61*r)

    return t_v

def compute_virtual_potential_temperature(t, rh, p, p0 = 1000):

    # Compute mixing ratio
    r = compute_mixing_ratio(rh, t, p)

    # Compute virtual temperature
    t_v = compute_virtual_temperature(t,r)

    # Gas constant for dry air
    Rd = 287 # J/(kg K)

    # Specific heat at constant pressure
    cp = 1005 # J/(kg K)

    # Compute the potential temperature
    vpt = t_v*(p0/p)**(Rd/cp)

    return vpt

def compute_obukhov_length(u, v, w, t, rh, p, p0 = 1000):

    # Von Karman constant
    k = 0.4

    # Gravitational acceleration
    g = 9.81

    # Compute friction velocity
    u_a = compute_friction_velocity(u, v, w)
    
    # Compute virtual potential temperature
    vpt = compute_virtual_potential_temperature(t, rh, p, p0)

    # Compute kinematic heat flux
    vpt_p = vpt - np.mean(vpt)
    w_p = w - np.mean(w)
    khf = np.mean(vpt_p * w_p)

    # Compute Obukhov length
    L = - (u_a**3 * np.mean(vpt)) / (k * g * khf)

    return L

def process_df_obukhov_length(df_min, df_s):
    
    df_s_aux = df_s.copy()
    df_s_aux.set_index("datetime", inplace=True)
    df_min_aux = df_min.set_index('datetime')

    groups = df_s_aux.groupby(pd.Grouper(freq="10min"))
    print(len(groups))
    
    df_out = pd.DataFrame()
    for time_bin, segment in groups:
        try:
            u = segment['u_40']
            v = segment['v_40']
            w = segment['w_40']
            t = segment['T_40']

            df_min_segment = df_min_aux.loc[time_bin]
            rh = df_min_segment['relativehumidity_34']
            p = df_min_segment['airpressure_21']

            L = compute_obukhov_length(u, v, w, t, rh, p)
            df_min_segment['L'] = L
            df_out = pd.concat([df_out,df_min_segment], axis=1)
        except:
            continue
    
    return df_out

In [164]:
df_min = pd.read_pickle('../02Data/10min_data')
df_s = pd.read_pickle('../02Data/10hz_data')
process_df_obukhov_length(df_min, df_s)

2


C:\Users\andreu.salcedo\AppData\Local\Temp\ipykernel_19504\3201758386.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_segment['L'] = L
C:\Users\andreu.salcedo\AppData\Local\Temp\ipykernel_19504\3201758386.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_segment['L'] = L
C:\Users\andreu.salcedo\AppData\Local\Temp\ipykernel_19504\3201758386.py:100: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat o

,2006-01-01
airpressure_21,991.000000
airpressure_92,-999.000000
airtemperature_101,-999.000000
airtemperature_34,-999.000000
airtemperature_42,6.690000
airtemperature_52,6.410000
airtemperature_72,6.160000
globalradiation_34,-999.000000
globalradiation_93,-999.000000
precipitation_101,-999.000000


In [123]:
dt = datetime(2006,1,1,0,0,0)
df_min[df_min['datetime'] == dt]

,airpressure_21,airpressure_92,airtemperature_101,airtemperature_34,airtemperature_42,airtemperature_52,airtemperature_72,globalradiation_34,globalradiation_93,precipitation_101,...,windspeed_41,windspeed_51,windspeed_61,windspeed_71,windspeed_81,windspeed_91,windspeed_42,windspeed_62,windspeed_82,datetime
0,991.0,-999,-999.0,-999.0,6.69,6.41,6.16,-999,-999.0,-999,...,11.24,11.38,11.44,11.69,11.74,11.92,0.0,0.0,0.0,2006-01-01


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'